In [95]:
import csv
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go2
import math
import zipfile
import pandas as pd
import statistics
from statistics import mean
from sklearn import feature_selection as fs
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [96]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [97]:
df_label = df['CreditScore']
df_data = df.drop('CreditScore', axis=1) 
df_data

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21
0,22,14,8557.39,797.12,8,6,22,2,37,15.00,...,7.00,0,1767.29,25.40,143,2,13.17,91.95,3.00,264.59
1,37,1,19718.92,1676.24,10,9,19,6,27,11.00,...,13.00,2,2057.56,28.64,197,2,64.07,107.67,3.00,285.89
2,33,13,32045.78,2677.48,6,9,30,7,10,10.00,...,11.00,2,1333.18,30.05,76,2,169.77,62.68,4.00,285.30
3,42,4,62976.28,5321.02,0,3,12,0,9,0.00,...,3.00,1,68.66,40.66,191,1,0.00,70.78,4.00,711.32
4,39,2,57818.72,4864.23,7,7,21,2,56,16.00,...,12.00,0,2348.77,37.88,174,2,73.71,395.14,3.00,307.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85265,23,6,76230.76,6159.56,6,10,31,3,35,15.00,...,14.00,2,1961.73,34.22,242,2,15026.00,176.55,1.00,539.75
85266,46,0,110607.09,8997.26,4,4,2,3,2,9.00,...,6.00,1,1174.05,24.78,265,1,154.82,232.18,5.00,752.72
85267,50,8,96275.84,8101.99,6,6,19,1,7,18.00,...,5.00,2,999.36,28.34,321,0,64.96,129.83,NaN,855.41
85268,55,9,69388.26,5543.35,2,4,6,0,8,1.00,...,2.00,1,819.98,40.50,266,1,0.00,184.21,1.00,640.12


standardize

In [98]:
"""
from sklearn.preprocessing import StandardScaler
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#norm_data = tempppp.copy()
scaler = StandardScaler()
scaler = scaler.fit(df_data.iloc[:,:17])
df_data.iloc[:,:17] = scaler.transform(df_data.iloc[:,:17])
df_data.describe()
"""


"\nfrom sklearn.preprocessing import StandardScaler\npd.set_option('display.float_format', lambda x: '%.2f' % x)\n#norm_data = tempppp.copy()\nscaler = StandardScaler()\nscaler = scaler.fit(df_data.iloc[:,:17])\ndf_data.iloc[:,:17] = scaler.transform(df_data.iloc[:,:17])\ndf_data.describe()\n"

In [99]:
"""
#scaler = StandardScaler()
pd.set_option('display.float_format', lambda x: '%.2f' % x)
scaler = scaler.fit(df_test.iloc[:,:17])
df_test.iloc[:,:17] = scaler.transform(df_test.iloc[:,:17])
df_test.describe()
"""

"\n#scaler = StandardScaler()\npd.set_option('display.float_format', lambda x: '%.2f' % x)\nscaler = scaler.fit(df_test.iloc[:,:17])\ndf_test.iloc[:,:17] = scaler.transform(df_test.iloc[:,:17])\ndf_test.describe()\n"

standardize end

In [100]:
from xgboost import XGBClassifier ,cv ,DMatrix
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=1000 , stop=2000 , num=10)]
max_depth = [int(x) for x in np.linspace(1,5,num=5)]
max_depth.append(None)
learning_rate = [round(float(x),2) for x in np.linspace(start=0.01 , stop=0.2 ,num=10)]
colsample_bytree = [round(float(x),) for x in np.linspace(start=0.5 , stop=0.2 , num=10)]
gamma = [round(float(x),2) for x in np.linspace(start=0 , stop=1 , num=5)]

random_grid = {
    'n_estimators' : n_estimators,
    'max_depth': max_depth,
    'learning_rate':learning_rate,
    'colsample_bytree':colsample_bytree,
    'gamma':gamma
}

In [101]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weight = compute_sample_weight(
    class_weight = 'balanced',
    y=df_label
)
xg4 = XGBClassifier(random_state = 42 , objective='multi:softmax')

xg_random = RandomizedSearchCV(estimator= xg4 , param_distributions=random_grid,
                                n_iter=50 , cv=4, verbose=1,random_state=42,n_jobs=4)
xg_random.fit(df_data,df_label)
xg_random.best_params_

Fitting 4 folds for each of 50 candidates, totalling 200 fits


{'n_estimators': 1555,
 'max_depth': 4,
 'learning_rate': 0.2,
 'gamma': 0.0,
 'colsample_bytree': 0}

In [102]:
outcome = xg_random.predict(df_test)
outcome_df = pd.DataFrame(outcome)
outcome_df1 = outcome_df.set_axis(["Label"], axis="columns")
outcome_df1.head()

,Label
0,2
1,0
2,0
3,0
4,0


In [103]:
outcome_df1.to_csv('ans1.csv',index_label='Id')